# Multi-agent recruiting workflow 
Last tested with letta version `0.5.3`

## Section 0: Setup a MemGPT client 

In [2]:
from letta import create_client 

client = create_client() 

In [3]:
from letta import LLMConfig, EmbeddingConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini")) 
client.set_default_embedding_config(EmbeddingConfig.default_config("text-embedding-ada-002")) 

## Section 1: Shared Memory Block 
Each agent will have both its own memory, and shared memory. The shared memory will contain information about the organization that the agents are all a part of. If one agent updates this memory, the changes will be propaged to the memory of all the other agents. 

In [4]:
from letta.schemas.block import Block 

org_description = "The company is called AgentOS " \
+ "and is building AI tools to make it easier to create " \
+ "and deploy LLM agents."

org_block = Block(label="company", value=org_description )

In [5]:
org_block

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.', limit=2000, template_name=None, template=False, label='company', description=None, metadata_={}, user_id=None, id='block-f212d9e6-f930-4d3b-b86a-40879a38aec4')

In [6]:
from letta.schemas.memory import BasicBlockMemory

class OrgMemory(BasicBlockMemory): 

    def __init__(self, persona: str, org_block: Block): 
        persona_block = Block(label="persona", value=persona)
        super().__init__(blocks=[persona_block, org_block])
        

## Section 2: Orchestrating Multiple Agents 
We'll implement a recruiting workflow that involves evaluating an candidate, then if the candidate is a good fit, writing a personalized email on the human's behalf. Since this task involves multiple stages, sometimes breaking the task down to multiple agents can improve performance (though this is not always the case). We will break down the task into: 

1. `eval_agent`: This agent is responsible for evaluating candidates based on their resume
2. `outreach_agent`: This agent is responsible for writing emails to strong candidates
3. `recruiter_agent`: This agent is responsible for generating leads from a database 

Much like humans, these agents will communicate by sending each other messages. We can do this by giving agents that need to communicate with other agents access to a tool that allows them to message other agents. 

#### Evaluator Agent
This agent will have tools to: 
* Read a resume 
* Submit a candidate for outreach (which sends the candidate information to the `outreach_agent`)

In [7]:
def read_resume(self, name: str): 
    """
    Read the resume data for a candidate given the name

    Args: 
        name (str): Candidate name 

    Returns: 
        resume_data (str): Candidate's resume data 
    """
    import os
    filepath = os.path.join("data", "resumes", name.lower().replace(" ", "_") + ".txt")
    #print("read", filepath)
    return open(filepath).read()

def submit_evaluation(self, candidate_name: str, reach_out: bool, resume: str, justification: str): 
    """
    Submit a candidate for outreach. 

    Args: 
        candidate_name (str): The name of the candidate
        reach_out (bool): Whether to reach out to the candidate
        resume (str): The text representation of the candidate's resume 
        justification (str): Justification for reaching out or not
    """
    from letta import create_client 
    client = create_client()
    message = "Reach out to the following candidate. " \
    + f"Name: {candidate_name}\n" \
    + f"Resume Data: {resume}\n" \
    + f"Justification: {justification}"
    # NOTE: we will define this agent later 
    if reach_out:
        response = client.send_message(
            agent_name="outreach_agent", 
            role="user", 
            message=message
        ) 
    else: 
        print(f"Candidate {candidate_name} is rejected: {justification}")

# TODO: add an archival andidate tool (provide justification) 

read_resume_tool = client.create_tool(read_resume) 
submit_evaluation_tool = client.create_tool(submit_evaluation)

In [9]:
skills = "Front-end (React, Typescript), software engineering " \
+ "(ideally Python), and experience with LLMs."
eval_persona = f"You are responsible to finding good recruiting " \
+ "candidates, for the company description. " \
+ f"Ideal canddiates have skills: {skills}. " \
+ "Submit your candidate evaluation with the submit_evaluation tool. "

# delete agent if exists 
if client.get_agent_id("eval_agent"): 
    client.delete_agent(client.get_agent_id("eval_agent"))

eval_agent = client.create_agent(
    name="eval_agent", 
    memory=OrgMemory(
        persona=eval_persona, 
        org_block=org_block,
    ), 
    tools=[read_resume_tool.name, submit_evaluation_tool.name]
)


#### Outreach agent 
This agent will email candidates with customized emails. Since sending emails is a bit complicated, we'll just pretend we sent an email by printing it in the tool call. 

In [10]:
def email_candidate(self, content: str): 
    """
    Send an email

    Args: 
        content (str): Content of the email 
    """
    print("Pretend to email:", content)
    return

email_candidate_tool = client.create_tool(email_candidate)

In [13]:
outreach_persona = "You are responsible for sending outbound emails " \
+ "on behalf of a company with the send_emails tool to " \
+ "potential candidates. " \
+ "If possible, make sure to personalize the email by appealing " \
+ "to the recipient with details about the company. " \
+ "You position is `Head Recruiter`, and you go by the name Bob, with contact info bob@gmail.com. " \
+ """
Follow this email template: 

Hi <candidate name>, 

<content> 

Best, 
<your name> 
<company name> 
"""


# delete agent if exists 
if client.get_agent_id("outreach_agent"): 
    client.delete_agent(client.get_agent_id("outreach_agent"))
    
outreach_agent = client.create_agent(
    name="outreach_agent", 
    memory=OrgMemory(
        persona=outreach_persona, 
        org_block=org_block
    ), 
    tools=[email_candidate_tool.name]
)

Next, we'll send a message from the user telling the `leadgen_agent` to evaluate a given candidate: 

In [14]:
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message="Candidate: Tony Stark"
)

In [15]:
response

LettaResponse(messages=[InternalMonologue(id='message-97a1ae82-f8f3-419f-94c4-263112dbc10b', date=datetime.datetime(2024, 11, 14, 1, 51, 26, 799617, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Checking the resume for Tony Stark to evaluate if he fits the bill for our needs.'), FunctionCallMessage(id='message-97a1ae82-f8f3-419f-94c4-263112dbc10b', date=datetime.datetime(2024, 11, 14, 1, 51, 26, 799617, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='read_resume', arguments='{\n  "name": "Tony Stark",\n  "request_heartbeat": true\n}', function_call_id='call_wOsiHlU3551JaApHKP7rK4Rt')), FunctionReturn(id='message-97a2b57e-40c6-4f06-a307-a0e3a00717ce', date=datetime.datetime(2024, 11, 14, 1, 51, 26, 803505, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "Failed",\n  "message": "Error calling function read_resume: [Errno 2] No such file or directory: \'data/resumes/tony_stark.txt\'",\n  "time": "2024-11-13 05:51:26 PM PST-0800"\n}', status='error', function_call_id='call_wOsiHlU3551JaApHKP7rK4Rt'), InternalMonologue(id='message-8e249aea-27ce-4788-b3e0-ac4c8401bc93', date=datetime.datetime(2024, 11, 14, 1, 51, 28, 360676, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="I couldn't retrieve Tony's resume. Need to handle this carefully to keep the conversation flowing."), FunctionCallMessage(id='message-8e249aea-27ce-4788-b3e0-ac4c8401bc93', date=datetime.datetime(2024, 11, 14, 1, 51, 28, 360676, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "It looks like I\'m having trouble accessing Tony Stark\'s resume at the moment. Can you provide more details about his qualifications?"\n}', function_call_id='call_1DoFBhOsP9OCpdPQjUfBcKjw')), FunctionReturn(id='message-5600e8e7-6c6f-482a-8594-a0483ef523a2', date=datetime.datetime(2024, 11, 14, 1, 51, 28, 361921, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:51:28 PM PST-0800"\n}', status='success', function_call_id='call_1DoFBhOsP9OCpdPQjUfBcKjw')], usage=LettaUsageStatistics(completion_tokens=103, prompt_tokens=4999, total_tokens=5102, step_count=2))

#### Providing feedback to agents 
Since MemGPT agents are persisted, we can provide feedback to agents that is used in future agent executions if we want to modify the future behavior. 

In [16]:
feedback = "Our company pivoted to foundation model training"
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message=feedback
)

In [17]:

feedback = "The company is also renamed to FoundationAI"
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message=feedback
)

In [18]:
response

LettaResponse(messages=[InternalMonologue(id='message-0adccea9-4b96-4cbb-b5fc-a9ef0120c646', date=datetime.datetime(2024, 11, 14, 1, 51, 34, 180327, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Updating the company name to reflect the rebranding. This is important for future candidate evaluations.'), FunctionCallMessage(id='message-0adccea9-4b96-4cbb-b5fc-a9ef0120c646', date=datetime.datetime(2024, 11, 14, 1, 51, 34, 180327, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='core_memory_replace', arguments='{\n  "label": "company",\n  "old_content": "The company has pivoted to foundation model training.",\n  "new_content": "The company is called FoundationAI and has pivoted to foundation model training.",\n  "request_heartbeat": true\n}', function_call_id='call_5s0KTElXdipPidchUu3R9CxI')), FunctionReturn(id='message-a2f278e8-ec23-4e22-a124-c21a0f46f733', date=datetime.datetime(2024, 11, 14, 1, 51, 34, 182291, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:51:34 PM PST-0800"\n}', status='success', function_call_id='call_5s0KTElXdipPidchUu3R9CxI'), InternalMonologue(id='message-91f63cb2-b544-4b2e-82b1-b11643df5f93', date=datetime.datetime(2024, 11, 14, 1, 51, 35, 841684, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Now I have the updated company info, time to check in on Tony.'), FunctionCallMessage(id='message-91f63cb2-b544-4b2e-82b1-b11643df5f93', date=datetime.datetime(2024, 11, 14, 1, 51, 35, 841684, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Got it, the new name is FoundationAI! What about Tony Stark\'s background catches your eye for this role? Any particular insights on his skills in front-end development or LLMs?"\n}', function_call_id='call_R4Erx7Pkpr5lepcuaGQU5isS')), FunctionReturn(id='message-813a9306-38fc-4665-9f3b-7c3671fd90e6', date=datetime.datetime(2024, 11, 14, 1, 51, 35, 842423, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:51:35 PM PST-0800"\n}', status='success', function_call_id='call_R4Erx7Pkpr5lepcuaGQU5isS')], usage=LettaUsageStatistics(completion_tokens=146, prompt_tokens=6372, total_tokens=6518, step_count=2))

In [19]:
response = client.send_message(
    agent_name="eval_agent", 
    role="system", 
    message="Candidate: Spongebob Squarepants"
)

In [20]:
client.get_core_memory(eval_agent.id).get_block("company")

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.\nThe company is called FoundationAI and has pivoted to foundation model training.', limit=2000, template_name=None, template=False, label='company', description=None, metadata_={}, user_id=None, id='block-f212d9e6-f930-4d3b-b86a-40879a38aec4')

In [21]:
client.get_core_memory(outreach_agent.id).get_block("company")

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.', limit=2000, template_name=None, template=False, label='company', description=None, metadata_={}, user_id=None, id='block-f212d9e6-f930-4d3b-b86a-40879a38aec4')

## Section 3: Adding an orchestrator agent 
So far, we've been triggering the `eval_agent` manually. We can also create an additional agent that is responsible for orchestrating tasks. 

In [22]:
#re-create agents 
client.delete_agent(eval_agent.id)
client.delete_agent(outreach_agent.id)

eval_agent = client.create_agent(
    name="eval_agent", 
    memory=OrgMemory(
        persona=eval_persona, 
        org_block=org_block,
    ), 
    tools=[read_resume_tool.name, submit_evaluation_tool.name]
)

outreach_agent = client.create_agent(
    name="outreach_agent", 
    memory=OrgMemory(
        persona=outreach_persona, 
        org_block=org_block
    ), 
    tools=[email_candidate_tool.name]
)

The `recruiter_agent` will be linked to the same `org_block` that we created before - we can look up the current data in `org_block` by looking up its ID: 

In [23]:
client.get_block(org_block.id)

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.\nThe company is called FoundationAI and has pivoted to foundation model training.', limit=2000, template_name=None, template=False, label='company', description=None, metadata_={}, user_id='user-00000000-0000-4000-8000-000000000000', id='block-f212d9e6-f930-4d3b-b86a-40879a38aec4')

In [25]:
from typing import Optional

def search_candidates_db(self, page: int) -> Optional[str]: 
    """
    Returns 1 candidates per page. 
    Page 0 returns the first 1 candidate, 
    Page 1 returns the next 1, etc.
    Returns `None` if no candidates remain. 

    Args: 
        page (int): The page number to return candidates from 

    Returns: 
        candidate_names (List[str]): Names of the candidates
    """
    
    names = ["Tony Stark", "Spongebob Squarepants", "Gautam Fang"]
    if page >= len(names): 
        return None
    return names[page]

def consider_candidate(self, name: str): 
    """
    Submit a candidate for consideration. 

    Args: 
        name (str): Candidate name to consider 
    """
    from letta import create_client 
    client = create_client()
    message = f"Consider candidate {name}" 
    print("Sending message to eval agent: ", message)
    response = client.send_message(
        agent_name="eval_agent", 
        role="user", 
        message=message
    ) 


# create tools 
search_candidate_tool = client.create_tool(search_candidates_db)
consider_candidate_tool = client.create_tool(consider_candidate)

# delete agent if exists 
if client.get_agent_id("recruiter_agent"): 
    client.delete_agent(client.get_agent_id("recruiter_agent"))

# create recruiter agent
recruiter_agent = client.create_agent(
    name="recruiter_agent", 
    memory=OrgMemory(
        persona="You run a recruiting process for a company. " \
        + "Your job is to continue to pull candidates from the " 
        + "`search_candidates_db` tool until there are no more " \
        + "candidates left. " \
        + "For each candidate, consider the candidate by calling "
        + "the `consider_candidate` tool. " \
        + "You should continue to call `search_candidates_db` " \
        + "followed by `consider_candidate` until there are no more " \
        " candidates. ",
        org_block=org_block
    ), 
    tools=[search_candidate_tool.name, consider_candidate_tool.name]
)
    


In [26]:
response = client.send_message(
    agent_name="recruiter_agent", 
    role="system", 
    message="Run generation"
)

In [27]:
response

LettaResponse(messages=[InternalMonologue(id='message-8c8ab238-a43e-4509-b7ad-699e9a47ed44', date=datetime.datetime(2024, 11, 14, 1, 52, 14, 780419, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='New user logged in. Excited to get started!'), FunctionCallMessage(id='message-8c8ab238-a43e-4509-b7ad-699e9a47ed44', date=datetime.datetime(2024, 11, 14, 1, 52, 14, 780419, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Welcome! I\'m thrilled to have you here. Let’s dive into what you need today!"\n}', function_call_id='call_2OIz7t3oiGsUlhtSneeDslkj')), FunctionReturn(id='message-26c3b7a3-51c8-47ae-938d-a3ed26e42357', date=datetime.datetime(2024, 11, 14, 1, 52, 14, 781455, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:52:14 PM PST-0800"\n}', status='success', function_call_id='call_2OIz7t3oiGsUlhtSneeDslkj')], usage=LettaUsageStatistics(completion_tokens=48, prompt_tokens=2398, total_tokens=2446, step_count=1))

In [28]:
client.delete_agent(eval_agent.id)
client.delete_agent(outreach_agent.id)

In [29]:
client.delete_agent(recruiter_agent.id)